<a href="https://colab.research.google.com/github/ak2742/mlplay/blob/Fine-Tuning/10)_Tagging_with_ollama_llama3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install package and load the extension
!pip install colab-xterm
%load_ext colabxterm
%xterm

# #- Install/run Ollama in terminal

# !curl -fsSL https://ollama.com/install.sh | sh
# !ollama serve & ollama pull llama3

In [ ]:
#@title Install libraries

#LangChain related libraries
!pip install -q -U langchain==0.1.20 langchain_experimental

In [ ]:
#@title tagging config

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

# tagging prompt template
tagging_prompt = ChatPromptTemplate.from_template(
"""
Extract the desired information from the  passage given in triple Backticks.

Only extract the properties mentioned in the 'Classification' function.

Please follow the response format as given.

Passage:
'''{input}'''
"""
)

# Output Schema for structured response
class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text. \
                           If no sentiment is detected, the value is 'Neutral'", enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10. \
            If no aggressiveness is detected, the value is 1"
    )
    language: str = Field(description="The language the text is written in \
            If unknown language, the value is 'Unknown'")


In [ ]:
#@title Create Chain with llama3

from langchain_experimental.llms.ollama_functions import OllamaFunctions

# Chain
llm = OllamaFunctions(model="llama3", format="json", temperature=0)
structured_llm = llm.with_structured_output(Classification)
chain = tagging_prompt | structured_llm

In [ ]:
#@title Get response in json format

inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
res = chain.invoke({"input": inp}).dict()
res